# Data
- Galaxy Zoo 2
- Table 1
- Accessible at https://data.galaxyzoo.org/

Plan:
1. Read GalaxyZoo file
2. Download Images from SDSS according to ra, dec from GZ2
3. Download Spectras

In [1]:
import pandas as pd
import numpy as np
import astropy
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
from scipy import interpolate
import requests
import html
import re

import math
import sys
import os
import subprocess
import astropy.io.fits as pyfits
from astroquery.sdss import SDSS
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS
import astropy.units as u

In [2]:
galaxy_zoo = fits.open('../../Data/GalaxyZoo2_fits_08.12.21/gz2_hart16.fits')

## Overview of file

In [3]:
galaxy_zoo.info()

Filename: ../../Data/GalaxyZoo2_fits_08.12.21/gz2_hart16.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU    547   239695R x 231C   [K, E, E, 44A, 44A, 80A, 8A, I, I, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K, I, E, E, E, D, K]   


In [4]:
galaxy_zoo_header = galaxy_zoo[0]

### Full data

Column description and format accessible at http://gz2hart.s3.amazonaws.com/gz2_hart16.txt

In [5]:
galaxy_zoo_data = Table(galaxy_zoo[1].data)
galaxy_zoo_data

dr7objid,ra,dec,rastring,decstring,sample,gz2_class,total_classifications,total_votes,t01_smooth_or_features_a01_smooth_count,t01_smooth_or_features_a01_smooth_weight,t01_smooth_or_features_a01_smooth_fraction,t01_smooth_or_features_a01_smooth_weighted_fraction,t01_smooth_or_features_a01_smooth_debiased,t01_smooth_or_features_a01_smooth_flag,t01_smooth_or_features_a02_features_or_disk_count,t01_smooth_or_features_a02_features_or_disk_weight,t01_smooth_or_features_a02_features_or_disk_fraction,t01_smooth_or_features_a02_features_or_disk_weighted_fraction,t01_smooth_or_features_a02_features_or_disk_debiased,t01_smooth_or_features_a02_features_or_disk_flag,t01_smooth_or_features_a03_star_or_artifact_count,t01_smooth_or_features_a03_star_or_artifact_weight,t01_smooth_or_features_a03_star_or_artifact_fraction,t01_smooth_or_features_a03_star_or_artifact_weighted_fraction,t01_smooth_or_features_a03_star_or_artifact_debiased,t01_smooth_or_features_a03_star_or_artifact_flag,t02_edgeon_a04_yes_count,t02_edgeon_a04_yes_weight,t02_edgeon_a04_yes_fraction,t02_edgeon_a04_yes_weighted_fraction,t02_edgeon_a04_yes_debiased,t02_edgeon_a04_yes_flag,t02_edgeon_a05_no_count,t02_edgeon_a05_no_weight,t02_edgeon_a05_no_fraction,t02_edgeon_a05_no_weighted_fraction,t02_edgeon_a05_no_debiased,t02_edgeon_a05_no_flag,t03_bar_a06_bar_count,t03_bar_a06_bar_weight,t03_bar_a06_bar_fraction,t03_bar_a06_bar_weighted_fraction,t03_bar_a06_bar_debiased,t03_bar_a06_bar_flag,t03_bar_a07_no_bar_count,t03_bar_a07_no_bar_weight,t03_bar_a07_no_bar_fraction,t03_bar_a07_no_bar_weighted_fraction,t03_bar_a07_no_bar_debiased,t03_bar_a07_no_bar_flag,t04_spiral_a08_spiral_count,t04_spiral_a08_spiral_weight,t04_spiral_a08_spiral_fraction,t04_spiral_a08_spiral_weighted_fraction,t04_spiral_a08_spiral_debiased,t04_spiral_a08_spiral_flag,t04_spiral_a09_no_spiral_count,t04_spiral_a09_no_spiral_weight,t04_spiral_a09_no_spiral_fraction,t04_spiral_a09_no_spiral_weighted_fraction,t04_spiral_a09_no_spiral_debiased,t04_spiral_a09_no_spiral_flag,t05_bulge_prominence_a10_no_bulge_count,t05_bulge_prominence_a10_no_bulge_weight,t05_bulge_prominence_a10_no_bulge_fraction,t05_bulge_prominence_a10_no_bulge_weighted_fraction,t05_bulge_prominence_a10_no_bulge_debiased,t05_bulge_prominence_a10_no_bulge_flag,t05_bulge_prominence_a11_just_noticeable_count,t05_bulge_prominence_a11_just_noticeable_weight,t05_bulge_prominence_a11_just_noticeable_fraction,t05_bulge_prominence_a11_just_noticeable_weighted_fraction,t05_bulge_prominence_a11_just_noticeable_debiased,t05_bulge_prominence_a11_just_noticeable_flag,t05_bulge_prominence_a12_obvious_count,t05_bulge_prominence_a12_obvious_weight,t05_bulge_prominence_a12_obvious_fraction,t05_bulge_prominence_a12_obvious_weighted_fraction,t05_bulge_prominence_a12_obvious_debiased,t05_bulge_prominence_a12_obvious_flag,t05_bulge_prominence_a13_dominant_count,t05_bulge_prominence_a13_dominant_weight,t05_bulge_prominence_a13_dominant_fraction,t05_bulge_prominence_a13_dominant_weighted_fraction,t05_bulge_prominence_a13_dominant_debiased,t05_bulge_prominence_a13_dominant_flag,t06_odd_a14_yes_count,t06_odd_a14_yes_weight,t06_odd_a14_yes_fraction,t06_odd_a14_yes_weighted_fraction,t06_odd_a14_yes_debiased,t06_odd_a14_yes_flag,t06_odd_a15_no_count,t06_odd_a15_no_weight,t06_odd_a15_no_fraction,t06_odd_a15_no_weighted_fraction,t06_odd_a15_no_debiased,t06_odd_a15_no_flag,t07_rounded_a16_completely_round_count,t07_rounded_a16_completely_round_weight,t07_rounded_a16_completely_round_fraction,t07_rounded_a16_completely_round_weighted_fraction,t07_rounded_a16_completely_round_debiased,t07_rounded_a16_completely_round_flag,t07_rounded_a17_in_between_count,t07_rounded_a17_in_between_weight,t07_rounded_a17_in_between_fraction,t07_rounded_a17_in_between_weighted_fraction,t07_rounded_a17_in_between_debiased,t07_rounded_a17_in_between_flag,t07_rounded_a18_cigar_shaped_count,t07_rounded_a18_cigar_shaped_weight,t07_rounded_a18_cigar_shaped_fraction,t07_rounded_a18_cigar_shaped_weighted_fract

In [6]:
# For checking single rows
galaxy_zoo_data[10]

dr7objid,ra,dec,rastring,decstring,sample,gz2_class,total_classifications,total_votes,t01_smooth_or_features_a01_smooth_count,t01_smooth_or_features_a01_smooth_weight,t01_smooth_or_features_a01_smooth_fraction,t01_smooth_or_features_a01_smooth_weighted_fraction,t01_smooth_or_features_a01_smooth_debiased,t01_smooth_or_features_a01_smooth_flag,t01_smooth_or_features_a02_features_or_disk_count,t01_smooth_or_features_a02_features_or_disk_weight,t01_smooth_or_features_a02_features_or_disk_fraction,t01_smooth_or_features_a02_features_or_disk_weighted_fraction,t01_smooth_or_features_a02_features_or_disk_debiased,t01_smooth_or_features_a02_features_or_disk_flag,t01_smooth_or_features_a03_star_or_artifact_count,t01_smooth_or_features_a03_star_or_artifact_weight,t01_smooth_or_features_a03_star_or_artifact_fraction,t01_smooth_or_features_a03_star_or_artifact_weighted_fraction,t01_smooth_or_features_a03_star_or_artifact_debiased,t01_smooth_or_features_a03_star_or_artifact_flag,t02_edgeon_a04_yes_count,t02_edgeon_a04_yes_weight,t02_edgeon_a04_yes_fraction,t02_edgeon_a04_yes_weighted_fraction,t02_edgeon_a04_yes_debiased,t02_edgeon_a04_yes_flag,t02_edgeon_a05_no_count,t02_edgeon_a05_no_weight,t02_edgeon_a05_no_fraction,t02_edgeon_a05_no_weighted_fraction,t02_edgeon_a05_no_debiased,t02_edgeon_a05_no_flag,t03_bar_a06_bar_count,t03_bar_a06_bar_weight,t03_bar_a06_bar_fraction,t03_bar_a06_bar_weighted_fraction,t03_bar_a06_bar_debiased,t03_bar_a06_bar_flag,t03_bar_a07_no_bar_count,t03_bar_a07_no_bar_weight,t03_bar_a07_no_bar_fraction,t03_bar_a07_no_bar_weighted_fraction,t03_bar_a07_no_bar_debiased,t03_bar_a07_no_bar_flag,t04_spiral_a08_spiral_count,t04_spiral_a08_spiral_weight,t04_spiral_a08_spiral_fraction,t04_spiral_a08_spiral_weighted_fraction,t04_spiral_a08_spiral_debiased,t04_spiral_a08_spiral_flag,t04_spiral_a09_no_spiral_count,t04_spiral_a09_no_spiral_weight,t04_spiral_a09_no_spiral_fraction,t04_spiral_a09_no_spiral_weighted_fraction,t04_spiral_a09_no_spiral_debiased,t04_spiral_a09_no_spiral_flag,t05_bulge_prominence_a10_no_bulge_count,t05_bulge_prominence_a10_no_bulge_weight,t05_bulge_prominence_a10_no_bulge_fraction,t05_bulge_prominence_a10_no_bulge_weighted_fraction,t05_bulge_prominence_a10_no_bulge_debiased,t05_bulge_prominence_a10_no_bulge_flag,t05_bulge_prominence_a11_just_noticeable_count,t05_bulge_prominence_a11_just_noticeable_weight,t05_bulge_prominence_a11_just_noticeable_fraction,t05_bulge_prominence_a11_just_noticeable_weighted_fraction,t05_bulge_prominence_a11_just_noticeable_debiased,t05_bulge_prominence_a11_just_noticeable_flag,t05_bulge_prominence_a12_obvious_count,t05_bulge_prominence_a12_obvious_weight,t05_bulge_prominence_a12_obvious_fraction,t05_bulge_prominence_a12_obvious_weighted_fraction,t05_bulge_prominence_a12_obvious_debiased,t05_bulge_prominence_a12_obvious_flag,t05_bulge_prominence_a13_dominant_count,t05_bulge_prominence_a13_dominant_weight,t05_bulge_prominence_a13_dominant_fraction,t05_bulge_prominence_a13_dominant_weighted_fraction,t05_bulge_prominence_a13_dominant_debiased,t05_bulge_prominence_a13_dominant_flag,t06_odd_a14_yes_count,t06_odd_a14_yes_weight,t06_odd_a14_yes_fraction,t06_odd_a14_yes_weighted_fraction,t06_odd_a14_yes_debiased,t06_odd_a14_yes_flag,t06_odd_a15_no_count,t06_odd_a15_no_weight,t06_odd_a15_no_fraction,t06_odd_a15_no_weighted_fraction,t06_odd_a15_no_debiased,t06_odd_a15_no_flag,t07_rounded_a16_completely_round_count,t07_rounded_a16_completely_round_weight,t07_rounded_a16_completely_round_fraction,t07_rounded_a16_completely_round_weighted_fraction,t07_rounded_a16_completely_round_debiased,t07_rounded_a16_completely_round_flag,t07_rounded_a17_in_between_count,t07_rounded_a17_in_between_weight,t07_rounded_a17_in_between_fraction,t07_rounded_a17_in_between_weighted_fraction,t07_rounded_a17_in_between_debiased,t07_rounded_a17_in_between_flag,t07_rounded_a18_cigar_shaped_count,t07_rounded_a18_cigar_shaped_weight,t07_rounded_a18_cigar_shaped_fraction,t07_rounded_a18_cigar_shaped_weighted_fract

In [7]:
coordinates = galaxy_zoo_data["dr7objid", "ra", "dec", "gz2_class"]

In [8]:
test_set = coordinates[:20]

In [9]:
data_set = coordinates

In [10]:
ids = coordinates["dr7objid"]
test_ids = ids[:20]

In [11]:
data_ids = ids

## Get Data

In [12]:
# file to log errors that occur when downloading images and spectra
errorfile = open("error_log.txt", "w")

### Get images

https://skyserver.sdss.org/dr16/SkyServerWS/ImgCutout/getjpeg?ra=224.5941&dec=-1.09&width=512&height=512

In [14]:
# Iterate through test set
for row in data_set:
    try:
        ra = row['ra']
        dec = row['dec']
        dr7objid = row['dr7objid']
        #print("Get image with ra=", ra, " and dec=", dec)

        # Where to get the image (SkyServer)
        url = html.unescape("https://skyserver.sdss.org/dr16/SkyServerWS/ImgCutout/getjpeg?ra={ra}&dec={dec}&width=512&height=512".format(ra=ra, dec=dec))
        image = requests.get(url)

        # open file in which image should be saved
        path = "C:\\Users\\korte\\Nextcloud\\Cognitive Systems\\3. Semester\\Individual Research Module\\Data\\photometric_jpeg_15.12.21\\" + "{dr7objid}_ra={ra}_dec={dec}.jpeg".format(dr7objid=dr7objid,ra=ra, dec=dec)

        with open(path,'wb') as file:
            file.write(image.content)
            file.close()
    except:
        print("Image for id ", dr7objid, " failed")
        error = "Image for id " + str(dr7objid) + " failed"
        errorfile.write(error)

Image for id  587741532248145945  failed
Image for id  587741491434815613  failed
Image for id  587739407320809518  failed
Image for id  587729159516323868  failed
Image for id  587739652645453927  failed
Image for id  587739827676053933  failed
Image for id  587739407878848799  failed
Image for id  588017604680482964  failed
Image for id  587739504477732884  failed
Image for id  587729227683528711  failed
Image for id  587726031691514192  failed
Image for id  587741601490993168  failed
Image for id  587726016684294160  failed
Image for id  588023670240968801  failed
Image for id  587722982834569455  failed
Image for id  588017978345062477  failed
Image for id  587731889501634624  failed
Image for id  588007004720005199  failed
Image for id  587742611339870231  failed
Image for id  587726100953432287  failed
Image for id  588010877681139727  failed
Image for id  588017702952239151  failed
Image for id  587731681189036307  failed
Image for id  587733604799217673  failed
Image for id  58

### Get spectra

In [15]:
sdss_path = 'https://data.sdss.org/sas/dr16/sdss/spectro/redux/26/spectra/'
boss_path = 'https://data.sdss3.org/sas/dr12/boss/spectro/redux/v5_7_0/spectra/'

In [16]:
#class_names = ['star','galaxy', 'QSO', 'AGN'] 
#class_names = ['star','galaxy'] 
## Queries for star, galaxy, quasar und AGN

In [17]:
query1 = "SELECT plate, mjd, min(fiberid) as fiberid, class FROM SpecObj WHERE class = 'galaxy' GROUP BY plate, mjd, class ORDER BY plate, mjd, class"
#query2 = "SELECT top 100 plate, mjd, min(fiberid) as fiberid, class FROM SpecObj WHERE class = 'galaxy' AND subClass != 'AGN' GROUP BY plate, mjd, class ORDER BY plate, mjd, class"

### Idea with ra, dec

In [18]:
"""
-- This query does a table JOIN between the imaging (PhotoObj) and spectra
-- (SpecObj) tables and includes the necessary columns in the SELECT to upload
-- the results to the DAS (Data Archive Server) for FITS file retrieval.

SELECT
   p.objID,p.ra,p.dec,p.z,
   p.run, p.rerun, p.camcol, p.field,
   s.specobjid, s.z,
   s.plate, s.mjd, s.fiberid, pdr.dr8objid
FROM PhotoObj AS p
   JOIN SpecObj AS s ON s.bestobjid = p.objID
   JOIN PhotoObjDR7 AS pdr ON pdr.dr8objid = p.objid
WHERE 
   pdr.dr7objid = ?
"""

'\n-- This query does a table JOIN between the imaging (PhotoObj) and spectra\n-- (SpecObj) tables and includes the necessary columns in the SELECT to upload\n-- the results to the DAS (Data Archive Server) for FITS file retrieval.\n\nSELECT\n   p.objID,p.ra,p.dec,p.u,p.g,p.r,p.i,p.z,\n   p.run, p.rerun, p.camcol, p.field,\n   s.specobjid, s.img, s.z,\n   s.plate, s.mjd, s.fiberid, pdr.dr8objid\nFROM PhotoObj AS p\n   JOIN SpecObj AS s ON s.bestobjid = p.objID\n   JOIN PhotoObjDR7 AS pdr ON pdr.dr8objid = p.objid\nWHERE \n   pdr.dr7objid = ?\n'

In [19]:
main_query = "SELECT p.objID,p.ra,p.dec,p.u,p.g,p.r,p.i,p.z, p.run, p.rerun, p.camcol, p.field, s.specobjid, s.img, s.z, s.plate, s.mjd, s.fiberid, pdr.dr8objid, pdr.dr7objid FROM PhotoObj AS p JOIN SpecObj AS s ON s.bestobjid = p.objID JOIN PhotoObjDR7 AS pdr ON pdr.dr8objid = p.objid WHERE"

In [20]:
queries = []

for id in data_ids:
    # Round coordinates, to find spectrum in this span
    add = " pdr.dr7objid={id}".format(id=id)
    query = main_query + add
    queries.append(query)

In [21]:
print("Downloading {len} spectra".format(len=len(queries)))

In [ ]:
for i in range(len(queries)):
    
    try:

        sdss = SDSS.query_sql(queries[i])

        dr7objid = int(sdss['dr7objid'])

        speclist  = open('speclist.txt', 'w')

        for plate, mjd, fiberid in zip(sdss['plate'],sdss['mjd'],sdss['fiberid']):

            speclist.write("%04d/spec-%04d-%d-%04d.fits \n" %(plate, plate, mjd, fiberid))

        speclist.close()

        with open('speclist.txt', 'r') as f:

            names = f.readlines()

            #print("reading: ", names)

        for item in names:

            name = item[:-2]
            url = sdss_path + name
            r = requests.get(url)

            target_file = "C:\\Users\\korte\\Nextcloud\\Cognitive Systems\\3. Semester\\Individual Research Module\\Data\\spectral_fits_25.12.21\\" + str(dr7objid) + name[5:]

            with open(target_file,'wb') as f:

                f.write(r.content)
    except: 
        print("Spectrum for id ", dr7objid, " failed")
        error = "Spectrum for id " + str(dr7objid) + " failed"
        errorfile.write(error)

C:\Users\korte\Anaconda3\lib\site-packages\astroquery\sdss\core.py:862: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  arr = np.atleast_1d(np.genfromtxt(io.BytesIO(response.content),


Spectrum for id  587745403080146952  failed
Spectrum for id  587741600954712088  failed
Spectrum for id  587736546311274638  failed
Spectrum for id  587739829274411045  failed
Spectrum for id  587739827662290999  failed
Spectrum for id  587739810498412721  failed
Spectrum for id  587733603732291636  failed
Spectrum for id  587739407874719751  failed
Spectrum for id  587739406804058294  failed
Spectrum for id  587726101491679322  failed
Spectrum for id  587726101491679322  failed
Spectrum for id  587726101491679322  failed
Spectrum for id  587741488761208838  failed
Spectrum for id  587741708338200629  failed
Spectrum for id  587736477596778665  failed
Spectrum for id  587736477596778665  failed
Spectrum for id  588013382727958586  failed
Spectrum for id  588013382727958586  failed
Spectrum for id  587736584436777018  failed
Spectrum for id  587736584436777018  failed
Spectrum for id  588017991236845602  failed
Spectrum for id  588017991236845602  failed
Spectrum for id  587742589330128

In [ ]:
# close the error log file
errorfile.close()